In [24]:
import json
import os
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta
import time
from filelock import FileLock

class NiftySummaryProcessor:
    def __init__(self, file_path: str):
        self.file_path = Path(file_path)
        self.lock_path = self.file_path.with_suffix('.lock')  # Lock file for synchronization
        self.current_time = datetime.now().replace(microsecond=0)
        self.today_date = self.current_time.date()
        self.cleanup_done = False

        # Perform initial cleanup of old records
        self.clean_old_records()

    def fetch_current_time(self):
        """Fetch the current system time."""
        self.current_time = datetime.now().replace(microsecond=0)
        self.today_date = self.current_time.date()
        return self.current_time

    def parse_file_content(self, content):
        """Parse multiple JSON objects from the file content."""
        decoder = json.JSONDecoder()
        records = []
        idx = 0
        while idx < len(content):
            try:
                obj, end = decoder.raw_decode(content, idx)
                records.append(obj)
                idx = end
            except json.JSONDecodeError:
                idx += 1  # Skip invalid characters
        return records

    def clean_old_records(self):
        """Delete JSON records with timestamps not from today, run only once."""
        if self.cleanup_done or not self.file_path.exists():
            return

        with FileLock(self.lock_path):
            with open(self.file_path, 'r') as f:
                content = f.read()

            records = self.parse_file_content(content)
            valid_records = []
            for record in records:
                try:
                    record_timestamp = datetime.strptime(record['timestamp'], '%d-%b-%Y %H:%M:%S')
                    if record_timestamp.date() == self.today_date:
                        valid_records.append(record)
                except (KeyError, ValueError):
                    continue

            if len(valid_records) < len(records):  # Only write if changes are made
                with open(self.file_path, 'w') as f:
                    for rec in valid_records:
                        json.dump(rec, f, indent=4)
                        f.write('\n')
                print(f"Cleaned old records. Kept {len(valid_records)} valid records for {self.today_date}.")

        self.cleanup_done = True

    def filter_today_records(self):
        """Read and filter records from today, with synchronization."""
        with FileLock(self.lock_path):
            self.fetch_current_time()
            print(f"Current today_date: {self.today_date}")

            if not self.file_path.exists():
                print("File does not exist.")
                return []

            with open(self.file_path, 'r') as f:
                content = f.read()

            if not content.strip():
                print("File is empty.")
                return []

            records = self.parse_file_content(content)
            valid_records = []
            for record in records:
                try:
                    record_timestamp = datetime.strptime(record['timestamp'], '%d-%b-%Y %H:%M:%S')
                    record_date = record_timestamp.date()
                    print(f"Processing record timestamp: {record_timestamp}, date: {record_date}")
                    if record_date == self.today_date:
                        valid_records.append(record)
                        print(f"Kept record for {record_timestamp}")
                    else:
                        print(f"Discarded record for {record_timestamp} (not {self.today_date})")
                except (KeyError, ValueError) as e:
                    print(f"Skipped invalid record due to error: {e}")

            return valid_records

    def calculate_pcr_for_intervals(self):
        """Calculate PCR and PCRVol for specific past minutes (1, 2, 3, 5, 10 min ago)."""
        valid_records = self.filter_today_records()
        if not valid_records:
            print("No valid records found for processing.")
            return pd.DataFrame(columns=['Interval', 'PCR', 'PCRVol', 'Status'])

        # Define intervals as minutes ago from current time
        intervals = {'1min ago': 1, '2min ago': 2, '3min ago': 3, '5min ago': 5, '10min ago': 10, '15min ago': 15}
        results = {label: {'PCR': None, 'PCRVol': None, 'Status': 'Not Present'} for label in intervals.keys()}
        current_time = self.current_time

        # Check each record against exact minute intervals
        for record in valid_records:
            record_time = datetime.strptime(record['timestamp'], '%d-%b-%Y %H:%M:%S')
            time_diff_minutes = int((current_time - record_time).total_seconds() / 60)  # Exact minutes difference

            for label, minutes_ago in intervals.items():
                if time_diff_minutes == minutes_ago:
                    ce_oi = record['CE']['totOI']
                    pe_oi = record['PE']['totOI']
                    ce_vol = record['CE']['totVol']
                    pe_vol = record['PE']['totVol']
                    results[label]['PCR'] = pe_oi / ce_oi if ce_oi != 0 else None
                    results[label]['PCRVol'] = pe_vol / ce_vol if ce_vol != 0 else None
                    results[label]['Status'] = 'Present'

        # Convert to DataFrame
        data = [
            {'Interval': label, 'PCR': results[label]['PCR'], 'PCRVol': results[label]['PCRVol'], 'Status': results[label]['Status']}
            for label in intervals.keys()
        ]
        return pd.DataFrame(data)

    def process_and_display(self):
        """Fetch current time, filter records, calculate PCR, and display results."""
        self.fetch_current_time()
        df = self.calculate_pcr_for_intervals()
        print(f"\nPCR Analysis as of {self.current_time.strftime('%d-%b-%Y %H:%M:%S')}:")
        print(df.to_string(index=False))

# Example usage
if __name__ == "__main__":
    file_path = r"nifty_summary.txt"
    processor = NiftySummaryProcessor(file_path)
    
    # Simulate real-time processing
    try:
        while True:
            processor.process_and_display()
            time.sleep(10)  # Check every 10 seconds
    except KeyboardInterrupt:
        print("Stopped by user.")

Current today_date: 2025-08-31
Processing record timestamp: 2025-08-31 23:09:00, date: 2025-08-31
Kept record for 2025-08-31 23:09:00
Processing record timestamp: 2025-08-31 23:10:00, date: 2025-08-31
Kept record for 2025-08-31 23:10:00
Processing record timestamp: 2025-08-31 23:12:00, date: 2025-08-31
Kept record for 2025-08-31 23:12:00
Processing record timestamp: 2025-08-31 23:13:00, date: 2025-08-31
Kept record for 2025-08-31 23:13:00

PCR Analysis as of 31-Aug-2025 00:08:11:
 Interval  PCR PCRVol      Status
 1min ago None   None Not Present
 2min ago None   None Not Present
 3min ago None   None Not Present
 5min ago None   None Not Present
10min ago None   None Not Present
15min ago None   None Not Present
Stopped by user.


# Preparing to display in excel

In [30]:
import json
import os
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta
import time
from filelock import FileLock
import win32com.client as win32
import pythoncom

class NiftySummaryProcessor:
    def __init__(self, input_file_path: str, output_excel_path: str = "PCR_excel_display.xlsx", offset_rows: int = 1, offset_cols: int = 1):
        self.input_file_path = Path(input_file_path)  # File to read data from
        self.output_excel_path = Path(output_excel_path)  # File to write Excel data to
        self.lock_path = self.input_file_path.with_suffix('.lock')  # Lock file for synchronization
        self.offset_rows = offset_rows
        self.offset_cols = offset_cols
        self.current_time = datetime.now().replace(microsecond=0)
        self.today_date = self.current_time.date()
        self.cleanup_done = False

        # Perform initial cleanup of old records from input file
        self.clean_old_records()

    def fetch_current_time(self):
        """Fetch the current system time."""
        self.current_time = datetime.now().replace(microsecond=0)
        self.today_date = self.current_time.date()
        return self.current_time

    def parse_file_content(self, content):
        """Parse multiple JSON objects from the file content."""
        decoder = json.JSONDecoder()
        records = []
        idx = 0
        while idx < len(content):
            try:
                obj, end = decoder.raw_decode(content, idx)
                records.append(obj)
                idx = end
            except json.JSONDecodeError:
                idx += 1  # Skip invalid characters
        return records

    def clean_old_records(self):
        """Delete JSON records with timestamps not from today, run only once."""
        if self.cleanup_done or not self.input_file_path.exists():
            return

        with FileLock(self.lock_path):
            with open(self.input_file_path, 'r') as f:
                content = f.read()

            records = self.parse_file_content(content)
            valid_records = []
            for record in records:
                try:
                    record_timestamp = datetime.strptime(record['timestamp'], '%d-%b-%Y %H:%M:%S')
                    if record_timestamp.date() == self.today_date:
                        valid_records.append(record)
                except (KeyError, ValueError):
                    continue

            if len(valid_records) < len(records):  # Only write if changes are made
                with open(self.input_file_path, 'w') as f:
                    for rec in valid_records:
                        json.dump(rec, f, indent=4)
                        f.write('\n')
                print(f"Cleaned old records. Kept {len(valid_records)} valid records for {self.today_date}.")

        self.cleanup_done = True

    def filter_today_records(self):
        """Read and filter records from today, with synchronization."""
        with FileLock(self.lock_path):
            self.fetch_current_time()
            print(f"Current today_date: {self.today_date}")

            if not self.input_file_path.exists():
                print("Input file does not exist.")
                return []

            with open(self.input_file_path, 'r') as f:
                content = f.read()

            if not content.strip():
                print("Input file is empty.")
                return []

            records = self.parse_file_content(content)
            valid_records = []
            for record in records:
                try:
                    record_timestamp = datetime.strptime(record['timestamp'], '%d-%b-%Y %H:%M:%S')
                    record_date = record_timestamp.date()
                    print(f"Processing record timestamp: {record_timestamp}, date: {record_date}")
                    if record_date == self.today_date:
                        valid_records.append(record)
                        print(f"Kept record for {record_timestamp}")
                    else:
                        print(f"Discarded record for {record_timestamp} (not {self.today_date})")
                except (KeyError, ValueError) as e:
                    print(f"Skipped invalid record due to error: {e}")

            return valid_records

    def calculate_pcr_for_intervals(self):
        """Calculate PCR and PCRVol for specific past minutes (1, 2, 3, 5, 10 min ago)."""
        valid_records = self.filter_today_records()
        if not valid_records:
            print("No valid records found for processing.")
            return pd.DataFrame(columns=['Interval', 'PCR', 'PCRVol', 'Status'])

        # Define intervals as minutes ago from current time
        intervals = {'1min ago': 1, '2min ago': 2, '3min ago': 3, '5min ago': 5, '10min ago': 10}
        results = {label: {'PCR': None, 'PCRVol': None, 'Status': 'Not Present'} for label in intervals.keys()}
        current_time = self.current_time

        # Check each record against exact minute intervals
        for record in valid_records:
            record_time = datetime.strptime(record['timestamp'], '%d-%b-%Y %H:%M:%S')
            time_diff_minutes = int((current_time - record_time).total_seconds() / 60)  # Exact minutes difference

            for label, minutes_ago in intervals.items():
                if time_diff_minutes == minutes_ago:
                    ce_oi = record['CE']['totOI']
                    pe_oi = record['PE']['totOI']
                    ce_vol = record['CE']['totVol']
                    pe_vol = record['PE']['totVol']
                    results[label]['PCR'] = pe_oi / ce_oi if ce_oi != 0 else None
                    results[label]['PCRVol'] = pe_vol / ce_vol if ce_vol != 0 else None
                    results[label]['Status'] = 'Present'

        # Convert to DataFrame for Excel writing
        data = [
            {'Interval': label, 'PCR': results[label]['PCR'], 'PCRVol': results[label]['PCRVol'], 'Status': results[label]['Status']}
            for label in intervals.keys()
        ]
        return pd.DataFrame(data)

    def write_using_com(self, df: pd.DataFrame, timestamp: str):
        """Write the DataFrame to an Excel sheet with the specified offset."""
        def safe_set_attr(obj_getter, attr, value, retries=5, delay=0.5):
            for i in range(retries):
                try:
                    obj = obj_getter()
                    setattr(obj, attr, value)
                    return
                except Exception as e:
                    if '0x800ac472' in str(e):  # Busy/RPC_S_CALLPENDING error
                        time.sleep(delay)
                    else:
                        raise
            print(f"Failed to set {attr} after {retries} attempts.")

        pythoncom.CoInitialize()
        excel = win32.gencache.EnsureDispatch("Excel.Application")
        excel.Visible = True
        excel.DisplayAlerts = False

        try:
            wb = None
            for open_wb in excel.Workbooks:
                if open_wb.FullName.lower() == str(self.output_excel_path).lower():
                    wb = open_wb
                    break
            
            if wb is None:
                if self.output_excel_path.exists():
                    wb = excel.Workbooks.Open(str(self.output_excel_path))
                else:
                    wb = excel.Workbooks.Add()
                    wb.SaveAs(str(self.output_excel_path))

            ws = wb.Worksheets(1)

            try:
                if ws.ProtectContents:
                    ws.Unprotect()
            except Exception as e:
                print(f"Warning: Couldn't unprotect sheet: {e}")
            
            # Start clearing from the specified offset
            start_cell = ws.Cells(self.offset_rows, self.offset_cols)
            last_cell = ws.Cells(ws.Rows.Count, ws.Columns.Count)
            ws.Range(start_cell, last_cell).ClearContents()

            # Write timestamp with offset
            timestamp_cell = ws.Cells(self.offset_rows, self.offset_cols)
            safe_set_attr(lambda: timestamp_cell, "Value", f"Data fetched at: {timestamp}")
            
            # Write "PCR" header
            pcr_header_cell = ws.Cells(self.offset_rows + 1, self.offset_cols)
            safe_set_attr(lambda: pcr_header_cell, "Value", "PCR")
            safe_set_attr(lambda: pcr_header_cell.Font, "Bold", True)

            # Write "Current Value" row
            current_value_row = self.offset_rows + 2
            current_value_cell = ws.Cells(current_value_row, self.offset_cols)
            safe_set_attr(lambda: current_value_cell, "Value", "Current Value")
            safe_set_attr(lambda: current_value_cell.Font, "Bold", True)

            # Get the latest PCR value (from the most recent record)
            latest_record = max(self.filter_today_records(), key=lambda x: datetime.strptime(x['timestamp'], '%d-%b-%Y %H:%M:%S'), default=None)
            latest_pcr = latest_record['PE']['totOI'] / latest_record['CE']['totOI'] if latest_record and latest_record['CE']['totOI'] != 0 else "N/A"
            current_value_data_cell = ws.Cells(current_value_row, self.offset_cols + 1)
            safe_set_attr(lambda: current_value_data_cell, "Value", latest_pcr)

            # Write interval headers and sentiment placeholders
            interval_headers = ['1min ago', '2min ago', '3min ago', '5min ago', '10min ago']
            for col_idx, header in enumerate(interval_headers, start=self.offset_cols + 2):
                cell = ws.Cells(current_value_row, col_idx)
                safe_set_attr(lambda c=cell: c, "Value", header)
                safe_set_attr(lambda c=cell: c.Font, "Bold", True)
                sentiment_cell = ws.Cells(current_value_row + 1, col_idx)
                safe_set_attr(lambda c=sentiment_cell: c, "Value", "Buying/Selling/Neutral")

            # Write "PCR Volume" row
            pcr_volume_row = self.offset_rows + 4
            pcr_volume_cell = ws.Cells(pcr_volume_row, self.offset_cols)
            safe_set_attr(lambda: pcr_volume_cell, "Value", "PCR Volume")
            safe_set_attr(lambda: pcr_volume_cell.Font, "Bold", True)

            # Get the latest PCRVol value
            latest_pcrvol = latest_record['PE']['totVol'] / latest_record['CE']['totVol'] if latest_record and latest_record['CE']['totVol'] != 0 else "N/A"
            pcr_volume_data_cell = ws.Cells(pcr_volume_row, self.offset_cols + 1)
            safe_set_attr(lambda: pcr_volume_data_cell, "Value", latest_pcrvol)

            # Write "incre" placeholder
            incre_cell = ws.Cells(pcr_volume_row + 1, self.offset_cols + 1)
            safe_set_attr(lambda: incre_cell, "Value", "incre")

            # AutoFit columns for better readability
            ws.Columns.AutoFit()

            print(f"PCR data written successfully to {self.output_excel_path} at {timestamp}.")

        except Exception as e:
            print(f"Error during Excel update: {e}")
            raise

        finally:
            excel.DisplayAlerts = True
            pythoncom.CoUninitialize()

    def process_and_display(self):
        """Fetch current time, filter records, calculate PCR, write to Excel, and display results."""
        self.fetch_current_time()
        df = self.calculate_pcr_for_intervals()
        timestamp = self.current_time.strftime('%d-%b-%Y %H:%M:%S')
        self.write_using_com(df, timestamp)
        print(f"\nPCR Analysis as of {timestamp}:")
        print(df.to_string(index=False))

# Example usage
if __name__ == "__main__":
    input_file_path = r"C:\Users\jaiad\Pro T\Final_draft1\nifty_summary.txt"
    output_excel_path = r"C:\Users\jaiad\Pro T\Final_draft1\banknifty_dashboard.xlsx"
    processor = NiftySummaryProcessor(input_file_path, output_excel_path, offset_rows=40, offset_cols=2)
    
    # Simulate real-time processing
    try:
        while True:
            processor.process_and_display()
            time.sleep(10)  # Check every 10 seconds
    except KeyboardInterrupt:
        print("Stopped by user.")

Current today_date: 2025-08-31
Processing record timestamp: 2025-08-31 23:09:00, date: 2025-08-31
Kept record for 2025-08-31 23:09:00
Processing record timestamp: 2025-08-31 23:10:00, date: 2025-08-31
Kept record for 2025-08-31 23:10:00
Processing record timestamp: 2025-08-31 23:12:00, date: 2025-08-31
Kept record for 2025-08-31 23:12:00
Processing record timestamp: 2025-08-31 23:13:00, date: 2025-08-31
Kept record for 2025-08-31 23:13:00
Current today_date: 2025-08-31
Processing record timestamp: 2025-08-31 23:09:00, date: 2025-08-31
Kept record for 2025-08-31 23:09:00
Processing record timestamp: 2025-08-31 23:10:00, date: 2025-08-31
Kept record for 2025-08-31 23:10:00
Processing record timestamp: 2025-08-31 23:12:00, date: 2025-08-31
Kept record for 2025-08-31 23:12:00
Processing record timestamp: 2025-08-31 23:13:00, date: 2025-08-31
Kept record for 2025-08-31 23:13:00
PCR data written successfully to C:\Users\jaiad\Pro T\Final_draft1\banknifty_dashboard.xlsx at 31-Aug-2025 00:12:1

In [44]:
import json
import os
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta
import time
from filelock import FileLock
import win32com.client as win32
import pythoncom

class NiftySummaryProcessor:
    def __init__(self, input_file_path: str, output_excel_path: str = "PCR_excel_display.xlsx", offset_rows: int = 1, offset_cols: int = 1):
        self.input_file_path = Path(input_file_path)  # File to read data from
        self.output_excel_path = Path(output_excel_path)  # File to write Excel data to
        self.lock_path = self.input_file_path.with_suffix('.lock')  # Lock file for synchronization
        self.offset_rows = offset_rows
        self.offset_cols = offset_cols
        self.current_time = datetime.now().replace(microsecond=0)
        self.today_date = self.current_time.date()
        self.cleanup_done = False

        # Perform initial cleanup of old records from input file
        self.clean_old_records()

    def fetch_current_time(self):
        """Fetch the current system time."""
        self.current_time = datetime.now().replace(microsecond=0)
        self.today_date = self.current_time.date()
        return self.current_time

    def parse_file_content(self, content):
        """Parse multiple JSON objects from the file content."""
        decoder = json.JSONDecoder()
        records = []
        idx = 0
        while idx < len(content):
            try:
                obj, end = decoder.raw_decode(content, idx)
                records.append(obj)
                idx = end
            except json.JSONDecodeError:
                idx += 1  # Skip invalid characters
        return records

    def clean_old_records(self):
        """Delete JSON records with timestamps not from today, run only once."""
        if self.cleanup_done or not self.input_file_path.exists():
            return

        with FileLock(self.lock_path):
            with open(self.input_file_path, 'r') as f:
                content = f.read()

            records = self.parse_file_content(content)
            valid_records = []
            for record in records:
                try:
                    record_timestamp = datetime.strptime(record['timestamp'], '%d-%b-%Y %H:%M:%S')
                    if record_timestamp.date() == self.today_date:
                        valid_records.append(record)
                except (KeyError, ValueError):
                    continue

            if len(valid_records) < len(records):  # Only write if changes are made
                with open(self.input_file_path, 'w') as f:
                    for rec in valid_records:
                        json.dump(rec, f, indent=4)
                        f.write('\n')
                print(f"Cleaned old records. Kept {len(valid_records)} valid records for {self.today_date}.")

        self.cleanup_done = True

    def filter_today_records(self):
        """Read and filter records from today, with synchronization."""
        with FileLock(self.lock_path):
            self.fetch_current_time()
            print(f"Current today_date: {self.today_date}")

            if not self.input_file_path.exists():
                print("Input file does not exist.")
                return []

            with open(self.input_file_path, 'r') as f:
                content = f.read()

            if not content.strip():
                print("Input file is empty.")
                return []

            records = self.parse_file_content(content)
            valid_records = []
            for record in records:
                try:
                    record_timestamp = datetime.strptime(record['timestamp'], '%d-%b-%Y %H:%M:%S')
                    record_date = record_timestamp.date()
                    print(f"Processing record timestamp: {record_timestamp}, date: {record_date}")
                    if record_date == self.today_date:
                        valid_records.append(record)
                        print(f"Kept record for {record_timestamp}")
                    else:
                        print(f"Discarded record for {record_timestamp} (not {self.today_date})")
                except (KeyError, ValueError) as e:
                    print(f"Skipped invalid record due to error: {e}")

            return valid_records

    def calculate_pcr_for_intervals(self):
        """Calculate PCR and PCRVol for specific past minutes (1, 2, 5, 10, 15 min ago)."""
        valid_records = self.filter_today_records()
        if not valid_records:
            print("No valid records found for processing.")
            return pd.DataFrame(columns=['Interval', 'PCR', 'PCRVol', 'Status'])

        # Define intervals as minutes ago from current time
        intervals = {'1min ago': 1, '2min ago': 2, '5min ago': 5, '10min ago': 10, '15min ago': 15}
        results = {label: {'PCR': None, 'PCRVol': None, 'Status': 'Not Present'} for label in intervals.keys()}
        current_time = self.current_time

        # Check each record against exact minute intervals
        for record in valid_records:
            record_time = datetime.strptime(record['timestamp'], '%d-%b-%Y %H:%M:%S')
            time_diff_minutes = int((current_time - record_time).total_seconds() / 60)  # Exact minutes difference

            for label, minutes_ago in intervals.items():
                if time_diff_minutes == minutes_ago:
                    ce_oi = record['CE']['totOI']
                    pe_oi = record['PE']['totOI']
                    ce_vol = record['CE']['totVol']
                    pe_vol = record['PE']['totVol']
                    results[label]['PCR'] = pe_oi / ce_oi if ce_oi != 0 else None
                    results[label]['PCRVol'] = pe_vol / ce_vol if ce_vol != 0 else None
                    results[label]['Status'] = 'Present'

        # Convert to DataFrame for Excel writing
        data = [
            {'Interval': label, 'PCR': results[label]['PCR'], 'PCRVol': results[label]['PCRVol'], 'Status': results[label]['Status']}
            for label in intervals.keys()
        ]
        return pd.DataFrame(data)

    def write_using_com(self, df: pd.DataFrame, timestamp: str):
        """Write the DataFrame to an Excel sheet with the specified offset in the requested format."""
        def safe_set_attr(obj_getter, attr, value, retries=5, delay=0.5):
            for i in range(retries):
                try:
                    obj = obj_getter()
                    setattr(obj, attr, value)
                    return
                except Exception as e:
                    if '0x800ac472' in str(e):  # Busy/RPC_S_CALLPENDING error
                        time.sleep(delay)
                    else:
                        raise
            print(f"Failed to set {attr} after {retries} attempts.")

        pythoncom.CoInitialize()
        excel = win32.gencache.EnsureDispatch("Excel.Application")
        excel.Visible = True
        excel.DisplayAlerts = False

        try:
            wb = None
            for open_wb in excel.Workbooks:
                if open_wb.FullName.lower() == str(self.output_excel_path).lower():
                    wb = open_wb
                    break
            
            if wb is None:
                if self.output_excel_path.exists():
                    wb = excel.Workbooks.Open(str(self.output_excel_path))
                else:
                    wb = excel.Workbooks.Add()
                    wb.SaveAs(str(self.output_excel_path))

            ws = wb.Worksheets(1)

            try:
                if ws.ProtectContents:
                    ws.Unprotect()
            except Exception as e:
                print(f"Warning: Couldn't unprotect sheet: {e}")
            
            # Start clearing from the specified offset
            start_cell = ws.Cells(self.offset_rows, self.offset_cols)
            last_cell = ws.Cells(ws.Rows.Count, ws.Columns.Count)
            ws.Range(start_cell, last_cell).ClearContents()
            
            # Write interval headers (centered above data)
            header_row = self.offset_rows + 1
            interval_headers = ['1 min', '2 min', '5 min', '10 min', '15 min']
            for col_idx, header in enumerate(interval_headers, start=self.offset_cols + 2):
                cell = ws.Cells(header_row, col_idx)
                safe_set_attr(lambda c=cell: c, "Value", header)
                # safe_set_attr(lambda c=cell: c.Font, "Bold", True)
                # Merge cells for centered effect (optional, adjust as needed)
                ws.Range(ws.Cells(header_row, col_idx), ws.Cells(header_row, col_idx)).HorizontalAlignment = -4108  # xlCenter

            # Write "PCR" row
            pcr_row = self.offset_rows + 2
            pcr_label_cell = ws.Cells(pcr_row, self.offset_cols)
            safe_set_attr(lambda: pcr_label_cell, "Value", "PCR")
            safe_set_attr(lambda: pcr_label_cell.Font, "Bold", True)
            
            current_pcr_cell = ws.Cells(pcr_row, self.offset_cols + 1)
            latest_record = max(self.filter_today_records(), key=lambda x: datetime.strptime(x['timestamp'], '%d-%b-%Y %H:%M:%S'), default=None)
            latest_pcr = latest_record['PE']['totOI'] / latest_record['CE']['totOI'] if latest_record and latest_record['CE']['totOI'] != 0 else "N/A"
            safe_set_attr(lambda: current_pcr_cell, "Value", "Current_value")
            safe_set_attr(lambda: current_pcr_cell.Offset(1, 1), "Value", latest_pcr)

            # Write PCR values for intervals
            for idx, interval in enumerate(['1min ago', '2min ago', '5min ago', '10min ago', '15min ago']):
                pcr_value = df.loc[df['Interval'] == interval, 'PCR'].iloc[0] if not df.empty and interval in df['Interval'].values else "N/A"
                cell = ws.Cells(pcr_row, self.offset_cols + 2 + idx)  # Start at col_idx + 2 (after Current_value)
                safe_set_attr(lambda c=cell: c, "Value", pcr_value)

            # Write "PCR Vol" row
            pcr_vol_row = self.offset_rows + 3
            pcr_vol_label_cell = ws.Cells(pcr_vol_row, self.offset_cols)
            safe_set_attr(lambda: pcr_vol_label_cell, "Value", "PCR Vol")
            safe_set_attr(lambda: pcr_vol_label_cell.Font, "Bold", True)
            
            current_pcrvol_cell = ws.Cells(pcr_vol_row, self.offset_cols + 1)
            latest_pcrvol = latest_record['PE']['totVol'] / latest_record['CE']['totVol'] if latest_record and latest_record['CE']['totVol'] != 0 else "N/A"
            safe_set_attr(lambda: current_pcrvol_cell, "Value", "Current_volumne")
            safe_set_attr(lambda: current_pcrvol_cell.Offset(1, 1), "Value", latest_pcrvol)

            # Write PCRVol values for intervals
            for idx, interval in enumerate(['1min ago', '2min ago', '5min ago', '10min ago', '15min ago']):
                pcrvol_value = df.loc[df['Interval'] == interval, 'PCRVol'].iloc[0] if not df.empty and interval in df['Interval'].values else "N/A"
                cell = ws.Cells(pcr_vol_row, self.offset_cols + 2 + idx)  # Start at col_idx + 2 (after Current_volumne)
                safe_set_attr(lambda c=cell: c, "Value", pcrvol_value)

            # AutoFit columns for better readability
            ws.Columns.AutoFit()

            print(f"PCR data written successfully to {self.output_excel_path} at {timestamp}.")

        except Exception as e:
            print(f"Error during Excel update: {e}")
            raise

        finally:
            excel.DisplayAlerts = True
            pythoncom.CoUninitialize()

    def process_and_display(self):
        """Fetch current time, filter records, calculate PCR, write to Excel, and display results."""
        self.fetch_current_time()
        df = self.calculate_pcr_for_intervals()
        timestamp = self.current_time.strftime('%d-%b-%Y %H:%M:%S')
        self.write_using_com(df, timestamp)
        print(f"\nPCR Analysis as of {timestamp}:")
        print(df.to_string(index=False))

# Example usage
if __name__ == "__main__":
    input_file_path = r"C:\Users\jaiad\Pro T\Final_draft1\nifty_summary.txt"
    output_excel_path = r"C:\Users\jaiad\Pro T\Final_draft1\banknifty_dashboard.xlsx"
    processor = NiftySummaryProcessor(input_file_path, output_excel_path, offset_rows=40, offset_cols=2)
    
    # Simulate real-time processing
    try:
        while True:
            processor.process_and_display()
            time.sleep(10)  # Check every 10 seconds
    except KeyboardInterrupt:
        print("Stopped by user.")

Current today_date: 2025-08-31
Processing record timestamp: 2025-08-31 01:01:00, date: 2025-08-31
Kept record for 2025-08-31 01:01:00
Processing record timestamp: 2025-08-31 01:17:00, date: 2025-08-31
Kept record for 2025-08-31 01:17:00
Processing record timestamp: 2025-08-31 01:20:00, date: 2025-08-31
Kept record for 2025-08-31 01:20:00
Processing record timestamp: 2025-08-31 02:21:00, date: 2025-08-31
Kept record for 2025-08-31 02:21:00
Current today_date: 2025-08-31
Processing record timestamp: 2025-08-31 01:01:00, date: 2025-08-31
Kept record for 2025-08-31 01:01:00
Processing record timestamp: 2025-08-31 01:17:00, date: 2025-08-31
Kept record for 2025-08-31 01:17:00
Processing record timestamp: 2025-08-31 01:20:00, date: 2025-08-31
Kept record for 2025-08-31 01:20:00
Processing record timestamp: 2025-08-31 02:21:00, date: 2025-08-31
Kept record for 2025-08-31 02:21:00
PCR data written successfully to C:\Users\jaiad\Pro T\Final_draft1\banknifty_dashboard.xlsx at 31-Aug-2025 01:21:4